# Galaxy Decomposition

In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [3]:
figure_setup()
settings = Settings()

In [4]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

#### The Decomposition Phase-Space

In [5]:
def read_data(simulation: str) -> tuple:
    """
    This method returns the circularity and the normalized potential for the
    stars in the main object of the simulation.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The circularity and normalized potential.
    """
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    s.add_reference_to_potential()
    s.add_normalized_potential()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    return (s.circularity[is_real_star & is_main_obj],
            s.normalized_potential[is_real_star & is_main_obj],)

### Figures

In [33]:
def plot_phase_space_for_galaxy(simulation: str) -> None:

    _, _, _, snapshot = parse(simulation)

    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    ax.tick_params(which='both', direction="in")
    ax.set_axisbelow(True)
    ax.grid(True, ls='-', lw=0.25, c='silver')

    ax.set_xlim(-1.6, 1.6)
    ax.set_xticks([-1, 0, 1])
    ax.set_xlabel(r'$\epsilon = j_z \, j_\mathrm{circ}^{-1}$')

    ax.set_ylim(-1, 0)
    ax.set_yticks([-1, -.8, -.6, -.4, -.2, 0])
    ax.set_ylabel(r'$\tilde{e} = e \, \left| e \right|_\mathrm{max}^{-1}$')

    galaxy, _, _, _ = parse(simulation)
    label = f"Au{galaxy}"
    circularity, potential = read_data(simulation=simulation)

    _, _, _, _ = ax.hist2d(
        circularity, potential,
        cmap='nipy_spectral', bins=200,
        range=[ax.get_xlim(), ax.get_ylim()],
        norm=mcolors.LogNorm(vmin=1E0, vmax=1E3), rasterized=True)
    ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
            size=8.0, transform=ax.transAxes, ha='left', va='top')

    # Draw lines for the components
    ax.plot([DISC_MIN_CIRC] * 2, ax.get_ylim(),
            ls="-", c='k', lw=.75)
    # ax.plot([DISC_STD_CIRC - COLD_DISC_DELTA_CIRC] * 2, ax.get_ylim(),
    #         ls="--", c='k', lw=.75)
    # ax.plot([DISC_STD_CIRC + COLD_DISC_DELTA_CIRC] * 2, ax.get_ylim(),
    #         ls="--", c='k', lw=.75)
    ax.plot([ax.get_xlim()[0], DISC_MIN_CIRC], [BULGE_MAX_SPECIFIC_ENERGY] * 2,
            ls="-", c='k', lw=.75)
    
    fig.savefig(
        f"../images/galaxy_decomposition/phase_space_{simulation}{SUFFIX}.pdf")
    plt.close(fig)

In [34]:
plot_phase_space_for_galaxy("au9_or_l4_s127")

In [ ]:
# fig = plt.figure(figsize=(7, 8))
# gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
# axs = gs.subplots(sharex=True, sharey=True)

# for ax in axs.flatten():
#     ax.tick_params(which='both', direction="in")
#     if ax == axs[-1, -1]: ax.axis("off")
#     ax.set_xlim(-1.6, 1.6)
#     ax.set_xticks([-1, 0, 1])
#     ax.set_ylim(-1, 0)
#     ax.set_yticks([-.8, -.6, -.4, -.2, 0])
#     ax.set_axisbelow(True)
#     if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
#         ax.set_xlabel(r'$\epsilon = j_z \, j_\mathrm{circ}^{-1}$')
#         ax.tick_params(labelbottom=True)
#     if ax.get_subplotspec().is_first_col():
#         ax.set_ylabel(r'$\tilde{e} = e \, \left| e \right|_\mathrm{max}^{-1}$')

# for i in range(len(settings.groups["Included"])):
#     ax = axs.flatten()[i]
#     galaxy = settings.groups["Included"][i]
#     simulation = f"au{galaxy}_or_l4_s127"
#     label = f"Au{galaxy}"
#     circularity, potential = read_data(simulation=simulation)
#     _, _, _, im = ax.hist2d(circularity,
#                             potential,
#                             cmap='nipy_spectral',
#                             bins=200,
#                             range=[ax.get_xlim(), ax.get_ylim()],
#                             norm=mcolors.LogNorm(vmin=1E0, vmax=1E3),
#                             rasterized=True)
#     ax.text(x=0.05, y=0.95,
#             s=r"$\texttt{" + label + "}$",
#             size=6.0, transform=ax.transAxes,
#             ha='left', va='top',
#             )

#     # Draw lines for the components
#     ax.plot([DISC_MIN_CIRC] * 2,
#             ax.get_ylim(), ls="--", c='k', lw=0.75)
#     ax.plot([DISC_STD_CIRC - COLD_DISC_DELTA_CIRC] * 2,
#             ax.get_ylim(), ls="--", c='k', lw=0.75)
#     ax.plot([DISC_STD_CIRC + COLD_DISC_DELTA_CIRC] * 2,
#             ax.get_ylim(), ls="--", c='k', lw=0.75)
#     ax.plot([ax.get_xlim()[0], DISC_MIN_CIRC],
#             [BULGE_MAX_SPECIFIC_ENERGY] * 2,
#             ls="--", c='k', lw=0.75)

#     fig.savefig(
#         f"../images/galaxy_decomposition/phase_space_included{SUFFIX}.pdf")

# plt.close(fig)